In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_data = pd.read_csv(train_file_path, sep="\t",header=None)
test_data = pd.read_csv(test_file_path, sep="\t",header=None)
train_data.tail

In [ ]:
train_data[0][train_data[0] == 'ham'] = 0
train_data[0][train_data[0] == 'spam'] = 1
test_data[0][test_data[0] == 'ham'] = 0
test_data[0][test_data[0] == 'spam'] = 1

train_labels = train_data[0]
train_data = train_data[1]

test_labels = test_data[0]
test_data = test_data[1]


In [ ]:
print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)

In [ ]:
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = str(text).lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocab:
      code = vocab[word]  
      encoding.append(code) 
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding
tr_data = []
for i in train_data:
  tr_data.append(one_hot_encoding(i))

train_data = tr_data.copy()

te_data = []
for i in test_data:
  te_data.append(one_hot_encoding(i))

test_data = te_data.copy()

In [ ]:
train_data

In [ ]:
from keras.utils import pad_sequences

train_data = pad_sequences(np.array(train_data), 250).astype(np.int32)
test_data = pad_sequences(np.array(test_data), 250).astype(np.int32)

In [ ]:
train_data[10]

In [ ]:
len(vocab)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(vocab)+1000, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()


In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])



In [ ]:
history = model.fit(train_data.astype(np.int32), train_labels.astype(np.int32), epochs=10, validation_split=0.2)

In [ ]:
print(test_data.shape)
print(test_labels.shape)

In [ ]:
results = model.evaluate(test_data.astype(np.int32), test_labels.astype(np.int32))
print(results)

In [ ]:
def encode_text(text):
  w = np.array(one_hot_encoding(text))
  w= w.reshape((1,) + w.shape)
  return pad_sequences(w,250)
text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])



def predict_message(pred_text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred[0]) 
  # print(result[0])


  return (prediction)

pred_text = "100"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
